In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
pd.pandas.set_option('display.max_columns',None)
import matplotlib.pyplot as plt


In [ ]:
Train = pd.read_csv(r"../input/house-prices-data/train.csv",na_values=[np.nan,'NA','NaN'])
Test = pd.read_csv(r"../input/house-prices-data/test.csv",na_values=[np.nan,'NA','NaN'])

In [ ]:
y_train = Train['SalePrice']

In [ ]:
X = Train.copy()
X.head()

In [ ]:
X = pd.concat([X,Train],axis=0)

In [ ]:
Temporal_feature = [feat for feat in X.columns if 'Yr' in feat or 'Year' in feat]
Temporal_feature

In [ ]:
Temporal_feature_nan = [feat for feat in Temporal_feature if X[feat].isnull().sum()>0]
Temporal_feature_nan

In [ ]:
print(X['GarageYrBlt'].mode())
print(X['GarageYrBlt'].median())
print(X['GarageYrBlt'].mean())

In [ ]:
X['GarageYrBlt'] = X['GarageYrBlt'].fillna(X['GarageYrBlt'].median())

In [ ]:
X['GarageYrBlt'].isnull().sum()

In [ ]:
numerical_feature = [feat for feat in X.columns if X[feat].dtypes!='O' and feat not in Temporal_feature]
len(numerical_feature)

In [ ]:
numerical_feature_nan = [feat for feat in numerical_feature if X[feat].isnull().sum()>0]
numerical_feature_nan

In [ ]:
X[numerical_feature_nan] = X[numerical_feature_nan].fillna(X[numerical_feature_nan].median())

In [ ]:
X[numerical_feature_nan].isnull().sum()

In [ ]:
categorical_feature = [feat for feat in X.columns if X[feat].dtypes=='O']
len(categorical_feature)

In [ ]:
categorical_feature_nan = [feat for feat in categorical_feature if X[feat].isnull().sum()>0]
categorical_feature_nan

In [ ]:
# X.drop('Id',axis=1,inplace=True)
X.shape

In [ ]:
categorical_feature_nan_drop = [feat for feat in categorical_feature_nan if X[feat].isnull().sum()/len(X)>=0.7]
categorical_feature_nan_drop

In [ ]:
X.drop(categorical_feature_nan_drop,axis=1,inplace=True)

In [ ]:
categorical_feature = [feat for feat in X.columns if X[feat].dtypes=='O']
len(categorical_feature)

In [ ]:
X['MasVnrType'] = X['MasVnrType'].fillna( X['MasVnrType'].value_counts().index[0])
categorical_feature_nan = [feat for feat in categorical_feature if X[feat].isnull().sum()>0]
categorical_feature_nan

In [ ]:
for i in categorical_feature_nan:
    X[i].fillna(X[i].value_counts().index[0],inplace=True)

In [ ]:
X[categorical_feature_nan].isnull().sum()

In [ ]:
# X.drop('Id',axis=1,inplace=True)
X.head()

In [ ]:
df = pd.DataFrame(pd.get_dummies(X[categorical_feature],drop_first=True))
df.head()

In [ ]:
X.drop(categorical_feature,axis=1,inplace=True)

In [ ]:
X.shape

In [ ]:
X = pd.concat([X,df],axis=1)

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
Cormat = X.corr()
Set = set()
for i in range(len(Cormat.columns)):
    for j in range(i):
        if np.abs(Cormat.iloc[i,j])>0.85:
            col = Cormat.columns[i]
            Set.add(col)

In [ ]:
sb.heatmap(Cormat)

In [ ]:
Set

In [ ]:
X.drop(Set,axis=1,inplace=True)

In [ ]:
X.shape

In [ ]:
Train['SaleType'].value_counts()

In [ ]:
variance = [feat for feat in X.columns if np.var(X[feat])<0.01]
variance

In [ ]:
X.drop(variance,axis=1,inplace=True)

In [ ]:
X.shape

In [ ]:
Train.shape[0]

In [ ]:
X_train = X[:Train.shape[0]]

In [ ]:
X_train.shape

In [ ]:
X_test = X[Train.shape[0]:]

In [ ]:
X_test.shape

In [ ]:
from sklearn.feature_selection import f_classif

In [ ]:
feature_fvalue = f_classif(X_train,y_train)[1]
feature_fvalue

In [ ]:
feature = X_train.columns[feature_fvalue<0.05]
feature

In [ ]:
X_train = X_train[feature]
X_test = X_test[feature]

In [ ]:
print(X_train.shape,X_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X_train = pd.DataFrame(StandardScaler().fit_transform(X_train))
X_train.head()
X_test = pd.DataFrame(StandardScaler().fit_transform(X_test))
X_test.head()

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lrg=LinearRegression()

In [ ]:
lrg.fit(X_train,y_train)

In [ ]:
LR = pd.Series(lrg.predict(X_test))

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score,make_scorer

In [ ]:
print(cross_val_score(lrg,X_train,y_train,cv=5,scoring = make_scorer(r2_score)))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
print(np.mean(cross_val_score(RandomForestRegressor(n_estimators=200),X_train,y_train,cv=5,scoring = make_scorer(r2_score))))

In [ ]:
from sklearn.svm import SVC

In [ ]:
print(np.mean(cross_val_score(SVC(),X_train,y_train,cv=5,scoring = make_scorer(r2_score))))

In [ ]:
from xgboost import XGBRegressor

In [ ]:
print(np.mean(cross_val_score(XGBRegressor(learning_rate = 0.11),X_train,y_train,cv=5,scoring = make_scorer(r2_score))))

In [ ]:
RRT = RandomForestRegressor(n_estimators=200)

In [ ]:
RRT.fit(X_train,y_train)

In [ ]:
RT =pd.Series(RRT.predict(X_test))

In [ ]:
DATA = pd.concat([LR,RT],axis=1)

In [ ]:
DATA.columns = ['LR','RT']
DATA.head()

In [ ]:
DATA['diff'] = DATA.LR-DATA.RT

In [ ]:
DATA.head()

In [ ]:
sb.heatmap(DATA.corr(),annot=True)

In [ ]:
#So by looking at it Linear regression give 1 r2 score and xgboost gives me 0.997 which is really good